In [5]:
import pandas as pd
import numpy as np

import re
import os

os.chdir('..')
os.chdir('results')

In [30]:
def combined_df(dataset='FordA', model_name='ResidualCNN', reg=True, exp_name=None):

    attack_type = 'fgsm_disc_attack' if reg else 'fgsm_attack'
    exp = '_' + exp_name if exp_name else ''
    
    path = '/'.join([dataset, model_name, 'attack', attack_type + exp])

    files = os.listdir(path)
    files = [f for f in files if f.endswith('.csv')]

    pattern = r"(?<==).*?(?=.csv)"

    all_res = pd.DataFrame()

    for file in files:
        alpha = float(re.search(pattern, file)[0])
        curr_res = pd.read_csv(path + '/' + file, index_col=0)

        curr_res['alpha'] = alpha
        curr_res.columns = [column.strip(' ') for column in curr_res.columns] 

        all_res = pd.concat([all_res, curr_res], axis=0)

    return all_res.reset_index(drop=True)


def aggregated_results(
    dataset='FordA', 
    model_name='ResidualCNN', 
    reg=True, 
    exp_name='', 
    k=1, 
    metr='F_EFF_CONC',
    metr_max=True
    ):

    all_df = combined_df(dataset=dataset, model_name=model_name, reg=reg, exp_name=exp_name)

    grouped = all_df.groupby(['eps', 'alpha'])
    result = grouped.apply(lambda x: x.nlargest(k, metr, keep='last') if metr_max else x.nsmallest(k, metr, keep='last'))

    return result

In [33]:
aggregated_results(exp_name="MODE")

ACC  ROC        PR   F1  EFF          L1  ACC_CORRECT  \
eps  alpha                                                                 
0.03 0.001    254  0.0  0.0  0.484091  0.0  1.0  227.691116          0.0   
     0.010    101  0.0  0.0  0.484091  0.0  1.0  229.804352          0.0   
     0.100    50   0.0  0.0  0.484091  0.0  1.0  232.031296          0.0   
     1.000    152  0.0  0.0  0.484091  0.0  1.0  232.983612          0.0   
     1000.000 203  0.0  0.0  0.484091  0.0  1.0  233.203033          0.0   

                   ACC_ORIG_ADV  ROUGHNESS  ROUGHNESS_NORM  PROB_HID  \
eps  alpha                                                             
0.03 0.001    254      0.083333   0.747931        7.963537       1.0   
     0.010    101      0.083333   0.786598        8.375248       1.0   
     0.100    50       0.083333   0.819173        8.722080       1.0   
     1.000    152      0.083333   0.831864        8.857211       1.0   
     1000.000 203      0.083333   0.832962        8.868900       1.0   

                   ACC_DISC  F1_DISC  ROC_AUC_DISC  CONC  F_EFF_CONC  step_id  \
eps  alpha                                                                      
0.03 0.001    254       1.0      1.0           1.0   0.0         0.0     50.0   
     0.010    101       1.0      1.0           1.0   0.0         0.0     50.0   
     0.100    50        1.0      1.0           1.0   0.0         0.0     50.0   
     1.000    152       1.0      1.0           1.0   0.0         0.0     50.0   
     1000.000 203       1.0      1.0           1.0   0.0         0.0     50.0   

                    eps     alpha  
eps  alpha                         
0.03 0.001    254  0.03     0.001  
     0.010    101  0.03     0.010  
     0.100    50   0.03     0.100  
     1.000    152  0.03     1.000  
     1000.000 203  0.03  1000.000